In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os 
os.chdir("/content/drive/My Drive/Colab Notebooks/ATML Project/Gutenberg_English_Fiction_1k/NEW/")

In [ ]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import tokenize
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.util import ngrams
import gensim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import os
import codecs
import re
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import Counter
from collections import OrderedDict
# import nlp
import spacy
import en_core_web_sm
from datetime import datetime
from sklearn.model_selection import GridSearchCV
import sklearn.utils as utils
from sklearn.svm import SVC
from sklearn import preprocessing

nlp = en_core_web_sm.load()
nlp.max_length = 3050000

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm



#Importing original features extracted

In [ ]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ATML Project/Gutenberg_English_Fiction_1k/final_features_without_null_values.csv", encoding="ISO-8859-1", delimiter=",")
data

,postive,neutral,negative,sentence_length,TTR,person_count,she_pronoun,he_pronoun,ease_of_readability,class
0,0.121,0.777,0.102,5048,0.858659,104,68,309,83.40,2
1,0.154,0.765,0.081,911,0.866100,17,54,127,78.15,5
2,0.170,0.738,0.092,8891,0.864052,201,313,392,90.04,5
3,0.113,0.803,0.085,8798,0.876325,280,162,408,94.47,8
4,0.172,0.703,0.125,1065,0.864347,19,75,98,79.83,5
...,...,...,...,...,...,...,...,...,...,...
989,0.152,0.763,0.085,19591,0.865487,480,456,572,79.90,5
990,0.133,0.780,0.087,6036,0.877662,235,197,322,80.37,5
991,0.112,0.800,0.088,5498,0.860010,193,86,365,78.89,2
992,0.134,0.752,0.114,5370,0.873383,124,145,219,84.28,2


##Splitting Training and Test data using Stratified Sampling

In [ ]:
x_true = data.drop(['class'], axis=1)
y_true = data[['class']]
x_train, x_test, y_train, y_test = train_test_split(x_true, y_true, stratify = y_true, test_size = 0.2)
print("Number of samples in training data :", x_train.shape[0])
print("Number of samples in test data :", x_test.shape[0])

# Checking total no. of data points for each class in train and test data

unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

unique1, counts1 = np.unique(y_test, return_counts=True)
print(dict(zip(unique1, counts1)))

Number of samples in training data : 795
Number of samples in test data : 199
{0: 2, 1: 4, 2: 89, 3: 5, 4: 5, 5: 633, 6: 14, 7: 29, 8: 14}
{1: 1, 2: 22, 3: 1, 4: 1, 5: 159, 6: 4, 7: 7, 8: 4}


##Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
print(x_test)

[[ 0.73910583 -1.40506348  1.45043776 ...  2.01610586  0.54810638
   0.28456303]
 [-0.02300719 -0.05432465  0.12879938 ... -0.99470761 -0.41228008
  -1.080144  ]
 [-0.47130896 -0.51641951  1.57630809 ... -0.40210305  0.40856305
  -0.43180404]
 ...
 [-0.82995038  1.43859721 -1.38164449 ... -0.89912623  0.11305953
   0.027726  ]
 [ 1.05291707 -1.12069741  0.50641035 ...  1.91096634  0.62198227
   0.16100359]
 [-0.47130896  0.7632278  -0.68935772 ...  0.78310606  0.54810638
   0.22347747]]


##Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
import sklearn.utils as utils
from sklearn.svm import SVC
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
# Performing CV to tune parameters for best SVM fit 
y = utils.validation.column_or_1d(y_train, warn=False)
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(x_train, y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
# reference : https://github.com/krishnaik06/GRIDSearchCV/blob/master/Gridsearchcv.ipynb
# View the accuracy score
print('Best score for training data:', svm_model.best_score_,"\n") 
# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C) 
print('Best Kernel:',svm_model.best_estimator_.kernel)
print('Best Gamma:',svm_model.best_estimator_.gamma)

final_model = svm_model.best_estimator_
y_pred = final_model.predict(x_test)

Best score for training data: 0.7962264150943398 

Best C: 1
Best Kernel: rbf
Best Gamma: 0.001


In [ ]:
print('\n *******************Evaluation Metrics (average = weighted) for test data original***************************** \n ')

print('F1 Score: ', metrics.f1_score(y_test, y_pred, average='weighted'))
print('Precision Score: ', metrics.precision_score(y_test, y_pred, average='weighted'))
print('Recall Score: ', metrics.recall_score(y_test, y_pred, average='weighted'))
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

print('\n *******************Evaluation Metrics (average = macro) for test data original***************************** \n ')

print('F1 Score: ', metrics.f1_score(y_test, y_pred, average='macro'))
print('Precision Score: ', metrics.precision_score(y_test, y_pred, average='macro'))
print('Recall Score: ', metrics.recall_score(y_test, y_pred, average='macro'))


 *******************Evaluation Metrics (average = weighted) for test data original***************************** 
 
F1 Score:  0.7097217933241626
Precision Score:  0.6383929698744981
Recall Score:  0.7989949748743719
Accuracy:  0.7989949748743719

 *******************Evaluation Metrics (average = macro) for test data original***************************** 
 
F1 Score:  0.11103351955307263
Precision Score:  0.09987437185929648
Recall Score:  0.125


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
